In [140]:
import pandas as pd
import re
! pip install nltk
import nltk
import numpy as np

In [141]:
dataset = pd.read_csv('AnnualReports16_processed.csv', encoding='latin-1')
# 计算每列的空缺值数量和百分比
missing_values = dataset.isnull().sum()
percentage_missing = (missing_values / len(dataset)) * 100

# 创建一个DataFrame来展示结果
missing_data_summary = pd.DataFrame({
    'Column Name': dataset.columns,
    'Missing Values': missing_values,
    'Percentage': percentage_missing
})

# 打印结果
print(missing_data_summary)

                                   Column Name  Missing Values  Percentage
cik                                        cik               0    0.000000
company_name                      company_name               0    0.000000
filed_date                          filed_date               0    0.000000
market_abnormal_return  market_abnormal_return               0    0.000000
nasdq                                    nasdq               0    0.000000
market_value                      market_value               0    0.000000
btm                                        btm             186    5.472198
pre_alpha                            pre_alpha               0    0.000000
pre_rmse                              pre_rmse               0    0.000000
InstOwn_Perc                      InstOwn_Perc               0    0.000000
log_share                            log_share               5    0.147102
processed_text                  processed_text             298    8.767284


In [142]:
# 删除包含空值的行
dataset_cleaned = dataset.dropna()

In [143]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [144]:
y = dataset_cleaned['market_abnormal_return']  # 因变量

In [145]:
lexicon_LM = pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2023.csv')
# 将'Word'列的所有值转换为小写
lexicon_LM['Word'] = lexicon_LM['Word'].str.lower()

lexicon_LM.head()


,Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Complexity,Syllables,Source
0,aardvark,1,664,2.690000e-08,1.860000e-08,4.050000e-06,131,0,0,0,0,0,0,0,0,2,12of12inf
1,aardvarks,2,3,1.210000e-10,8.230000e-12,9.020000e-09,1,0,0,0,0,0,0,0,0,2,12of12inf
2,abaci,3,9,3.640000e-10,1.110000e-10,5.160000e-08,7,0,0,0,0,0,0,0,0,3,12of12inf
3,aback,4,29,1.170000e-09,6.330000e-10,1.560000e-07,28,0,0,0,0,0,0,0,0,2,12of12inf
4,abacus,5,9349,3.790000e-07,3.830000e-07,3.460000e-05,1239,0,0,0,0,0,0,0,0,3,12of12inf


In [146]:
# 从词典中提取每个分类的词汇列表
Negative = set(lexicon_LM[lexicon_LM['Negative'] != 0]['Word'])
Positive = set(lexicon_LM[lexicon_LM['Positive'] != 0]['Word'])
Uncertainty = set(lexicon_LM[lexicon_LM['Uncertainty'] != 0]['Word'])
Litigious = set(lexicon_LM[lexicon_LM['Litigious'] != 0]['Word'])
Strong_Modal = set(lexicon_LM[lexicon_LM['Strong_Modal'] != 0]['Word'])
Weak_Modal = set(lexicon_LM[lexicon_LM['Weak_Modal'] != 0]['Word'])
Constraining = set(lexicon_LM[lexicon_LM['Constraining'] != 0]['Word'])
Complexity = set(lexicon_LM[lexicon_LM['Complexity'] != 0]['Word'])


# 打印每个分类的词汇数量
for category, words in dictionaries.items():
    print(f"{category}: {len(words)} words")

Negative: 2355 words
Positive: 354 words
Uncertainty: 297 words
Litigious: 905 words
Strong_Modal: 19 words
Weak_Modal: 27 words
Constraining: 184 words
Complexity: 53 words


In [147]:
def sentiment_score(text, sen_list):
    # 确保文本不为空或None
    if pd.isnull(text) or text == "":
        return 0
    # 计算文本中情感词汇的总数
    total_count = sum(text.lower().count(word) for word in sen_list if word in text.lower())
    # 归一化得分，使用文本长度作为分母
    return total_count / max(len(text), 1)

In [148]:
for sentiment, score_name in zip(
    [Negative, Positive, Uncertainty, Litigious, Strong_Modal, Weak_Modal, Constraining, Complexity],
    ['Negative_score', 'Positive_score', 'Uncertainty_score', 'Litigious_score', 'Strong_Modal_score', 'Weak_Modal_score', 'Constraining_score', 'Complexity_score']
):
    dataset_cleaned.loc[:, score_name] = dataset_cleaned['processed_text'].apply(lambda x: sentiment_score(x, list(sentiment)))

print(dataset_cleaned.head())

/var/folders/cx/8zplw71j7m955_00z36gftxw0000gn/T/ipykernel_1859/948644083.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_cleaned.loc[:, score_name] = dataset_cleaned['processed_text'].apply(lambda x: sentiment_score(x, list(sentiment)))
/var/folders/cx/8zplw71j7m955_00z36gftxw0000gn/T/ipykernel_1859/948644083.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_cleaned.loc[:, score_name] = dataset_cleaned['processed_text'].apply(lambda x: sentiment_score(x, list(sentiment)))
/var/folders

         cik               company_name  filed_date  market_abnormal_return  \
0   804212.0                 airgas inc  20160510.0               -0.008756   
1   880460.0  perfumania holdings, inc.  20160429.0               -0.004723   
2  1276591.0         hansen medical inc  20160425.0               -0.007461   
4    12659.0              h&r block inc  20160617.0               -0.001475   
5  1050825.0              steelcase inc  20160415.0               -0.051923   

   nasdq  market_value       btm  pre_alpha  pre_rmse  InstOwn_Perc  \
0      0      9.238859 -1.615560   0.168271  2.084189      0.846428   
1      1      3.546929  0.883620  -0.149365  4.478955      0.097400   
2      1      3.784871 -3.415026  -0.183296  7.257763      0.305207   
4      0      8.403640 -5.263362  -0.113956  1.942857      0.957540   
5      0      7.307156 -0.704711  -0.055984  1.959769      0.860600   

   log_share                                     processed_text  \
0   1.137248  item 7. managemen

/var/folders/cx/8zplw71j7m955_00z36gftxw0000gn/T/ipykernel_1859/948644083.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_cleaned.loc[:, score_name] = dataset_cleaned['processed_text'].apply(lambda x: sentiment_score(x, list(sentiment)))


In [151]:
X_LM = dataset_cleaned[['Negative_score', 'Positive_score', 'Uncertainty_score', 'Litigious_score', 
                        'Strong_Modal_score', 'Weak_Modal_score', 'Constraining_score', 'Complexity_score']]

In [155]:

# 拆分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_LM, y, test_size=0.2, random_state=42)

# 存储模型性能的字典
model_performance = {}

# 为每个特征建立一元线性回归模型
for feature in X_train.columns:
    # 为每个特征添加常数项
    X_train_feature = sm.add_constant(X_train[[feature]])
    X_test_feature = sm.add_constant(X_test[[feature]])

    # 构建模型并拟合
    model = sm.OLS(y_train, X_train_feature).fit()
    
     # 打印模型系数
    print(f"Coefficient for {feature}: {model.params[feature]}")


    # 预测测试集
    y_pred = model.predict(X_test_feature)

    # 评估模型性能
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    medae = median_absolute_error(y_test, y_pred)

    # 存储性能指标
    model_performance[feature] = {'MSE': mse, 'MAE': mae, 'R2': r2, 'MedAE': medae}

    # 打印性能指标
    print(f"Feature: {feature}")
    print(f"MSE: {mse}")
    print(f'MAE: {mae}')
    print(f'R²: {r2}')
    print(f'MedAE: {medae}')
    
    
    # 打印模型的统计摘要
    print(model.summary())


# 可以选择将模型性能的字典转换为DataFrame，便于查看
performance_df = pd.DataFrame(model_performance).T
print(performance_df)



Coefficient for Negative_score: -0.258827962746385
Feature: Negative_score
MSE: 0.0034150808938024873
MAE: 0.029838343131092748
R²: 0.0003254681452214969
MedAE: 0.016537874074686737
                              OLS Regression Results                              
Dep. Variable:     market_abnormal_return   R-squared:                       0.000
Model:                                OLS   Adj. R-squared:                 -0.000
Method:                     Least Squares   F-statistic:                    0.4835
Date:                    Sun, 28 Apr 2024   Prob (F-statistic):              0.487
Time:                            23:45:33   Log-Likelihood:                 3526.8
No. Observations:                    2335   AIC:                            -7050.
Df Residuals:                        2333   BIC:                            -7038.
Df Model:                               1                                         
Covariance Type:                nonrobust                              